In [1]:
                                                                                                                                                                                                                                                                                                                                                                                                                            import pandas as pd
import numpy as np
from sklearn.utils import shuffle

In [2]:
import ngram
import pickle
from helpers import *


In [3]:
data = pd.read_csv("../datasets/fake_or_real_news.csv", encoding='utf-8', )
data.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
data.head()

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
data.columns = ['Headline',  'articleBody', 'type']
data.head()

,Headline,articleBody,type
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
data['type'].value_counts()

REAL    3171
FAKE    3164
Name: type, dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 3 columns):
Headline       6335 non-null object
articleBody    6335 non-null object
type           6335 non-null object
dtypes: object(3)
memory usage: 148.6+ KB


In [8]:
type_rename = {'REAL': 'reliable', 'FAKE': 'unreliable'}
data['type'] = data['type'].apply(lambda x: type_rename[x])
data.reset_index(drop=True, inplace=True)

In [9]:
data.head()

,Headline,articleBody,type
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",unreliable
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,unreliable
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,reliable
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",unreliable
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,reliable


In [11]:
targets_dict = {'reliable': '0', 'unreliable': '1'}
data['target'] = list(map(lambda x: targets_dict[str(x)], data['type']))

In [18]:
data.type.value_counts()

reliable      3171
unreliable    3164
Name: type, dtype: int64

In [12]:
data.head(10)

,Headline,articleBody,type,target
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",unreliable,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,unreliable,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,reliable,0
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",unreliable,1
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,reliable,0
5,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",unreliable,1
6,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",unreliable,1
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,reliable,0
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,reliable,0
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,reliable,0


In [21]:
# To check for Kaggle dataset
targets = ['0', '1']
targets_dict = dict(zip(targets, ['reliable', 'unreliable']))
targets_dict

{'0': 'reliable', '1': 'unreliable'}

In [23]:
# For the other dataset
targets = ['0', '1']
targets_dict = dict(zip(targets, ))
targets_dict

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [24]:
{'reliable': '0', 'unreliable': '1'}

{'reliable': '0', 'unreliable': '1'}

In [25]:
data['target'] = list(map(lambda x: targets_dict[str(x)], data['type']))
data.shape

KeyError: 'unreliable'

In [16]:
data.head(10)

,Headline,articleBody,target,type
0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,unreliable
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,reliable
2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,unreliable
3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,unreliable
4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1,unreliable
5,Jackie Mason: Hollywood Would Love Trump if He...,"In these trying times, Jackie Mason is the Voi...",0,reliable
6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,Ever wonder how Britain’s most iconic pop pian...,1,unreliable
7,Benoît Hamon Wins French Socialist Party’s Pre...,"PARIS — France chose an idealistic, traditi...",0,reliable
8,Excerpts From a Draft Script for Donald Trump’...,Donald J. Trump is scheduled to make a highly ...,0,reliable
9,"A Back-Channel Plan for Ukraine and Russia, Co...",A week before Michael T. Flynn resigned as nat...,0,reliable


In [13]:
data.to_csv('../datasets/fake_or_real_processed.csv', index=False)

In [14]:
print ("generate unigram")
data["Headline_unigram"] = data["Headline"].map(lambda x: preprocess_data(x))
data["articleBody_unigram"] = data["articleBody"].map(lambda x: preprocess_data(x))

generate unigram


In [15]:
print ("generate bigram")
join_str = "_"
data["Headline_bigram"] = data["Headline_unigram"].map(lambda x: ngram.getBigram(x, join_str))
data["articleBody_bigram"] = data["articleBody_unigram"].map(lambda x: ngram.getBigram(x, join_str))

generate bigram


In [16]:
print ("generate trigram")
join_str = "_"
data["Headline_trigram"] = data["Headline_unigram"].map(lambda x: ngram.getTrigram(x, join_str))
data["articleBody_trigram"] = data["articleBody_unigram"].map(lambda x: ngram.getTrigram(x, join_str))

generate trigram


In [17]:
with open('../saved_data/kaggle/data_new.pkl', 'wb') as outfile:
    pickle.dump(data, outfile, -1)
print('dataframe saved in data_new.pkl')

dataframe saved in data_new.pkl


In [18]:
data.head(10)

,Headline,articleBody,type,target,Headline_unigram,articleBody_unigram,Headline_bigram,articleBody_bigram,Headline_trigram,articleBody_trigram
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",unreliable,1,"[smell, hillari, fear]","[daniel, greenfield, shillman, journal, fellow...","[smell_hillari, hillari_fear]","[daniel_greenfield, greenfield_shillman, shill...",[smell_hillari_fear],"[daniel_greenfield_shillman, greenfield_shillm..."
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,unreliable,1,"[watch, exact, moment, paul, ryan, commit, pol...","[googl, pinterest, digg, linkedin, reddit, stu...","[watch_exact, exact_moment, moment_paul, paul_...","[googl_pinterest, pinterest_digg, digg_linkedi...","[watch_exact_moment, exact_moment_paul, moment...","[googl_pinterest_digg, pinterest_digg_linkedin..."
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,reliable,0,"[kerri, go, pari, gestur, sympathi]","[secretari, state, john, kerri, said, monday, ...","[kerri_go, go_pari, pari_gestur, gestur_sympathi]","[secretari_state, state_john, john_kerri, kerr...","[kerri_go_pari, go_pari_gestur, pari_gestur_sy...","[secretari_state_john, state_john_kerri, john_..."
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",unreliable,1,"[berni, support, twitter, erupt, anger, dnc, t...","[kayde, king, kaydeek, novemb, 2016, lesson, t...","[berni_support, support_twitter, twitter_erupt...","[kayde_king, king_kaydeek, kaydeek_novemb, nov...","[berni_support_twitter, support_twitter_erupt,...","[kayde_king_kaydeek, king_kaydeek_novemb, kayd..."
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,reliable,0,"[battl, new, york, whi, primari, matter]","[primari, day, new, york, front, runner, hilla...","[battl_new, new_york, york_whi, whi_primari, p...","[primari_day, day_new, new_york, york_front, f...","[battl_new_york, new_york_whi, york_whi_primar...","[primari_day_new, day_new_york, new_york_front..."
5,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",unreliable,1,"[tehran, usa]","[immigr, grandpar, 50, year, ago, arriv, new, ...",[tehran_usa],"[immigr_grandpar, grandpar_50, 50_year, year_a...",[tehran_usa],"[immigr_grandpar_50, grandpar_50_year, 50_year..."
6,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",unreliable,1,"[girl, horrifi, watch, boyfriend, left, facetim]","[share, bayle, luciani, left, screenshot, bayl...","[girl_horrifi, horrifi_watch, watch_boyfriend,...","[share_bayle, bayle_luciani, luciani_left, lef...","[girl_horrifi_watch, horrifi_watch_boyfriend, ...","[share_bayle_luciani, bayle_luciani_left, luci..."
7,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,reliable,0,"[britain, schindler, die, 106]","[czech, stockbrok, save, 650, jewish, children...","[britain_schindler, schindler_die, die_106]","[czech_stockbrok, stockbrok_save, save_650, 65...","[britain_schindler_die, schindler_die_106]","[czech_stockbrok_save, stockbrok_save_650, sav..."
8,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,reliable,0,"[fact, check, trump, clinton, command, chief, ...","[hillari, clinton, donald, trump, made, inaccu...","[fact_check, check_trump, trump_clinton, clint...","[hillari_clinton, clinton_donald, donald_trump...","[fact_check_trump, check_trump_clinton, trump_...","[hillari_clinton_donald, clinton_donald_trump,..."
9,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,reliable,0,"[iran, report, make, new, push, uranium, conce...","[iranian, negoti, report, made, last, ditch, p...","[iran_report, report_make, make_new, new_push,...","[iranian_negoti, negoti_report, report_made, m...","[iran_report_make, rep

## The preprocessing and data pickle Ends Here!
### Next step>>> Get features.

## Generating Targets features for reading labels

In [6]:
with open('../saved_data/data_kaggle.pkl', 'rb') as infile:
    data = pickle.load(infile)

In [13]:
data.head(10)

,type,articleBody,Headline,target,Headline_unigram,articleBody_unigram,Headline_bigram,articleBody_bigram,Headline_trigram,articleBody_trigram
0,reliable,The Onion's violent satire painting fictional ...,"The Onion's Spoof Tweets Spark Anger, Fear; No...",0,"[onion, spoof, tweet, spark, anger, fear, laug...","[onion, violent, satir, paint, fiction, imag, ...","[onion_spoof, spoof_tweet, tweet_spark, spark_...","[onion_violent, violent_satir, satir_paint, pa...","[onion_spoof_tweet, spoof_tweet_spark, tweet_s...","[onion_violent_satir, violent_satir_paint, sat..."
1,unreliable,"By Tyler Durden / ZeroHedge Back in May, we fi...",“Here’s Why Current Situation is Even Worse Th...,1,"[whi, current, situat, even, wors, number, cri...","[tyler, durden, zerohedg, back, may, first, re...","[whi_current, current_situat, situat_even, eve...","[tyler_durden, durden_zerohedg, zerohedg_back,...","[whi_current_situat, current_situat_even, situ...","[tyler_durden_zerohedg, durden_zerohedg_back, ..."
2,unreliable,"In New York City, yesterday’s morning commute ...",What Is the World Coming to?,1,"[world, come]","[new, york, citi, yesterday, morn, commut, int...",[world_come],"[new_york, york_citi, citi_yesterday, yesterda...",[world_come],"[new_york_citi, york_citi_yesterday, citi_yest..."
3,unreliable,zerohedge.com / by Steve Saretsky via VanCityC...,Canadian Homeowners Take Out HELOCs To Fund Su...,1,"[canadian, homeown, take, heloc, fund, subprim...","[zerohedg, com, steve, saretski, via, vancityc...","[canadian_homeown, homeown_take, take_heloc, h...","[zerohedg_com, com_steve, steve_saretski, sare...","[canadian_homeown_take, homeown_take_heloc, ta...","[zerohedg_com_steve, com_steve_saretski, steve..."
4,unreliable,"In August number, Europe’s major powers threw ...",The Christmas Truce of World War I,1,"[christma, truce, world, war]","[august, number, europ, major, power, threw, t...","[christma_truce, truce_world, world_war]","[august_number, number_europ, europ_major, maj...","[christma_truce_world, truce_world_war]","[august_number_europ, number_europ_major, euro..."
5,unreliable,"Since CME unleashed its Bitcoin Futures, crypt...",Cryptos Slide: “Is There Any New Information T...,1,"[crypto, slide, ani, new, inform, exist, two, ...","[sinc, cme, unleash, bitcoin, futur, cryptocur...","[crypto_slide, slide_ani, ani_new, new_inform,...","[sinc_cme, cme_unleash, unleash_bitcoin, bitco...","[crypto_slide_ani, slide_ani_new, ani_new_info...","[sinc_cme_unleash, cme_unleash_bitcoin, unleas..."
6,unreliable,"Crazy Stupid Earth, Crazy Stupid Time we are l...",Transition Period Strategy and Policy Advice,1,"[transit, period, strategi, polici, advic]","[crazi, stupid, earth, crazi, stupid, time, li...","[transit_period, period_strategi, strategi_pol...","[crazi_stupid, stupid_earth, earth_crazi, craz...","[transit_period_strategi, period_strategi_poli...","[crazi_stupid_earth, stupid_earth_crazi, earth..."
7,unreliable,“People should not be afraid of their governme...,The Myth of Authority,1,"[myth, author]","[peopl, afraid, govern, govern, afraid, peopl,...",[myth_author],"[peopl_afraid, afraid_govern, govern_govern, g...",[myth_author],"[peopl_afraid_govern, afraid_govern_govern, go..."
8,reliable,A Federal Emergency Management Agency employe...,Judge Gives FEMA number Weeks to Change Policy...,0,"[judg, give, fema, number, week, chang, polici...","[feder, emerg, manag, agenc, employe, wait, ar...","[judg_give, give_fema, fema_number, number_wee...","[feder_emerg, emerg_manag, manag_agenc, agenc_...","[judg_give_fema, give_fema_number, fema_number...","[feder_emerg_manag, emerg_manag_agenc, manag_a..."
9,unreliable,"November number, number A city full of people ...",The Waitress,1,[waitress],"[novemb, number, number, citi, full, peopl, fa...",[waitress],"[novemb_number, number_number, number_citi, ci...",[waitress],"[novemb_number_number, number_number_citi, num..."


In [50]:
processed = data[['target', 'articleBody', 'Headline']]

In [37]:
processed.to_csv('../datasets/processed_fake_or_real_targets.csv', index_label='id')

In [19]:
targets = data[['Headline', 'target']]

In [20]:
targets.head(10)

,Headline,target
0,You Can Smell Hillary’s Fear,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,1
2,Kerry to go to Paris in gesture of sympathy,0
3,Bernie supporters on Twitter erupt in anger ag...,1
4,The Battle of New York: Why This Primary Matters,0
5,"Tehran, USA",1
6,Girl Horrified At What She Watches Boyfriend D...,1
7,‘Britain’s Schindler’ Dies at 106,0
8,Fact check: Trump and Clinton at the 'commande...,0
9,Iran reportedly makes new push for uranium con...,0


In [21]:
targets.to_csv('../datasets/targets_fake_or_real.csv', index_label='id')